In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.layers import Input,Dense, Dropout
from keras.models import Model,Sequential
from sklearn.metrics import roc_curve, auc
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_mutual_info_score,adjusted_rand_score,v_measure_score, completeness_score, homogeneity_score, silhouette_score,roc_curve, auc, f1_score, precision_recall_curve, precision_score,recall_score,accuracy_score,confusion_matrix
import sys
from sklearn.linear_model import LogisticRegression
sys.path.append('/Users/chenpete/Documents/ESP/practicum/autoencoders/code')
from sklearn.model_selection import KFold
from utils import * 
from sklearn.neighbors import BallTree
from sklearn.manifold import TSNE

Using TensorFlow backend.


### Link to Paper

https://arxiv.org/pdf/1809.10717.pdf

In [2]:
DATA_DIR = '/Users/PeterC/Documents/gt/practicum/autoencoders/data/'
DATA_DIR = '/Users/chenpete/Documents/ESP/practicum/autoencoders/data/'

non_scale_cols = ['time','y_early','y_rank','y']
param_cols = [
        'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9',
       'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19',
       'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29',
       'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39',
       'x40', 'x41', 'x42', 'x43', 'x44', 'x45', 'x46', 'x47', 'x48', 'x49',
       'x50', 'x51', 'x52', 'x53', 'x54', 'x55', 'x56', 'x57', 'x58', 'x59',
       'x60','x61']

y_col = 'y_early'
early_step = 4

df = pd.read_csv(DATA_DIR + 'paper_mill_rare_events.csv')
df['time'] = pd.to_datetime(df['time'])
X_train, X_test, y_train, y_test,y_rank_train, y_rank_test = preprocess_data(df,param_cols,y_col, non_scale_cols,early_step)

KeyError: "None of [Index(['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11',\n       'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21',\n       'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31',\n       'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39', 'x40', 'x41',\n       'x42', 'x43', 'x44', 'x45', 'x46', 'x47', 'x48', 'x49', 'x50', 'x51',\n       'x52', 'x53', 'x54', 'x55', 'x56', 'x57', 'x58', 'x59', 'x60', 'x61'],\n      dtype='object')] are in the [columns]"

# Model 1: Layer autoencoder

In [ ]:
model = simple_ann_autoencoder_regression(param_cols,32,16,32)
model.summary()

In [ ]:
history_callback = model.fit(X_train,X_train,epochs=75,batch_size = 32)
loss_history = history_callback.history['loss']

In [ ]:
model.save('autoencoder_32_16_32.h5')

In [ ]:
plt.plot(loss_history)
plt.title('loss history')

In [ ]:
X_pred_test = model.predict(X_test)

X_pred_train = model.predict(X_train)

df_results_test = results_df(X_test,X_pred_test,y_test,y_rank_test)
df_results_train = results_df(X_train,X_pred_train,y_train,y_rank_train)

In [ ]:
nominal_idx = y_rank_test==0
early1_idx = y_rank_test==1
early2_idx = y_rank_test==2
early3_idx = y_rank_test==3
early4_idx = y_rank_test==4



## Assess errors for each time lag

In [ ]:
res_nominal_train = df_results_train[df_results_train['y'] == 0]['residual_avg']

average_res_nominal_train = np.mean(res_nominal_train)
std_res_nominal_train = np.std(res_nominal_train)
threshold_3s = average_res_nominal_train + 3*std_res_nominal_train
threshold_5s = average_res_nominal_train + 5*std_res_nominal_train
threshold_max = np.max(res_nominal_train)

threshold_list = [('3s',threshold_3s),('5s',threshold_5s),('max',threshold_max)]

### Training residuals

In [ ]:
plt.figure()
sns.distplot(df_results_train['residual_avg'],norm_hist=True,kde=False,color='blue')


In [ ]:
for rank in range(1,np.max(df_results_test['y_rank'])+1):
    
    print('\n------------------ early time step of {}----------------------------'.format(rank))
    
    df_results_rank= df_results_test[(df_results_test['y_rank']==rank) | (df_results_test['y_rank']==0)]
    res_nominal_test = df_results_rank[df_results_rank['y'] == 0]['residual_avg']
    res_failures_test = df_results_rank[df_results_rank['y_rank'] == rank]['residual_avg']
    
    plt.figure()
    sns.distplot(res_failures_test,norm_hist=True,kde=False,color='red')
    sns.distplot(res_nominal_test,norm_hist=True,kde=False,color='blue')
    plt.legend(['failure residuals','non failure residuals'])
    plt.title('distribution of average residuals')

    y_true = np.array(df_results_rank['y'])
    y_pred_proba = np.array(df_results_rank['residual_avg']/np.max(df_results_rank['residual_avg']))

    plot_roc(y_true,y_pred_proba)
    
    for name,threshold in threshold_list:
        print('\n\n------ Using threshold {}---------------'.format(name))
        y_pred_threshold = np.where(df_results_rank['residual_avg'] > threshold, 1, 0)
        
        print('accuracy = {:.3f}'.format(accuracy_score(y_true,y_pred_threshold)))
        print('precision = {:.3f}'.format(precision_score(y_true,y_pred_threshold)))
        print('recall = {:.3f}'.format(recall_score(y_true,y_pred_threshold)))
        print('f1 = {:.3f}'.format(f1_score(y_true,y_pred_threshold)))


In [ ]:
sns.boxplot(x='y_rank',y='residual_avg',data=df_results_test)
plt.ylim((0,.75))

# Try different ways to analyze reconstruction error

In [ ]:
residuals_test = X_pred_test - X_test
residuals_train = X_pred_train - X_train

In [ ]:
residuals_test = residuals_test[nominal_idx |early1_idx]
y_test_early1 = y_test[nominal_idx |early1_idx]

In [ ]:
residuals_train.shape

In [ ]:
residuals_test.shape

### Try std error for early interval = 1

In [ ]:
residuals_train_mean = np.abs(np.mean(residuals_train,axis=1))
residuals_train_std = np.std(residuals_train,axis=1)

std_threshold = np.std(residuals_train_std) + 3*np.std(residuals_train_std)
std_threshold

In [ ]:
sns.distplot(residuals_train_std)

In [ ]:
residuals_test_mean = np.abs(np.mean(residuals_test,axis=1))
residuals_test_std = np.std(residuals_test,axis=1)

In [ ]:
residuals_test_std.shape

In [ ]:
plt.figure()
sns.distplot(residuals_test_std[early1_idx],norm_hist=True,kde=False,color='red')
sns.distplot(residuals_test_std[nominal_idx],norm_hist=True,kde=False,color='blue')
plt.legend(['failure residuals','non failure residuals'])
plt.title('distribution of std residual')

In [ ]:
y_pred_proba_std = residuals_test_std/np.max(residuals_test_std)

In [ ]:
plot_roc(y_test_early1,y_pred_proba_std)

In [ ]:
y_pred_threshold_std = np.where(residuals_test_std > std_threshold, 1, 0)

print('accuracy = {:.3f}'.format(accuracy_score(y_test_early1,y_pred_threshold_std)))
print('precision = {:.3f}'.format(precision_score(y_test_early1,y_pred_threshold_std)))
print('recall = {:.3f}'.format(recall_score(y_test_early1,y_pred_threshold_std)))
print('f1 = {:.3f}'.format(f1_score(y_test_early1,y_pred_threshold_std)))

### Look at all errors

## KNN on reconstruction error

In [ ]:
from sklearn.neighbors import DistanceMetric
#dm = DistanceMetric.get_metric('mahalanobis', V=np.cov(residuals_train))
tree = BallTree(residuals_train,metric ='euclidean')

In [ ]:
dist,ind = tree.query(residuals_test,k=2)
dist = np.mean(dist,axis=1)
dist = pd.Series(dist.ravel(),index=residuals_test.index)

In [ ]:
plt.figure()
sns.distplot(dist[early1_idx],norm_hist=True,kde=False,color='red')
sns.distplot(dist[nominal_idx],norm_hist=True,kde=False,color='blue')
plt.legend(['failure residuals','non failure residuals'])
plt.title('distribution of L2 distance for residual')

In [ ]:
y_pred_proba_knn = dist/np.max(dist)
plot_roc(y_test_early1,y_pred_proba_knn)

In [ ]:
y_pred_threshold_knn = np.where(dist > 9, 1, 0)
print('accuracy = {:.3f}'.format(accuracy_score(y_test_early1,y_pred_threshold_knn)))
print('precision = {:.3f}'.format(precision_score(y_test_early1,y_pred_threshold_knn)))
print('recall = {:.3f}'.format(recall_score(y_test_early1,y_pred_threshold_knn)))
print('f1 = {:.3f}\n\n'.format(f1_score(y_test_early1,y_pred_threshold_knn)))
print(confusion_matrix(y_test_early1,y_pred_threshold_knn))

# Cross Validation for KNN

In [ ]:
# K = 10
# X_train_list, X_test_list, y_train_list, y_test_list,y_rank_train_list, y_rank_test_list = preprocess_data_cv(df,param_cols,y_col, non_scale_cols,early_step, k = K)

In [ ]:
# for k in range(K):
    
#     X_train, y_train, y_rank_train = X_train_list[k],y_train_list[k],y_rank_train_list[k]
#     X_test, y_test, y_rank_test = X_test_list[k],y_test_list[k],y_rank_test_list[k]
    
#     nominal_idx = y_rank_test==0
#     early1_idx = y_rank_test==1
    
#     model = simple_ann_autoencoder_regression(param_cols,32,16,32)
#     history_callback = model.fit(X_train,X_train,epochs=75,batch_size = 32,verbose = 0)
    
#     X_pred_test = model.predict(X_test)
#     X_pred_train = model.predict(X_train)

#     df_results_test = results_df(X_test,X_pred_test,y_test,y_rank_test)
#     df_results_train = results_df(X_train,X_pred_train,y_train,y_rank_train)
    
#     residuals_test = X_pred_test - X_test
#     residuals_train = X_pred_train - X_train
    
#     residuals_test = residuals_test[nominal_idx |early1_idx]
#     y_test_early1 = y_test[nominal_idx |early1_idx]
    
#     tree = BallTree(residuals_train)
    
#     dist,ind = tree.query(residuals_test,k=1)
#     dist = pd.Series(dist.ravel(),index=residuals_test.index)
    
#     y_pred_proba_knn = dist/np.max(dist)
#     plot_roc(y_test_early1,y_pred_proba_knn)

# Try Logistic Regression on reconstruction error

In [ ]:
X_residual_train, X_residual_test, y_residual_train, y_residual_test = train_test_split(residuals_test, y_test_early1, test_size=0.2, random_state=0, stratify = y_test_early1)

clf = LogisticRegression()
clf.fit(X_residual_train,y_residual_train)
y_pred_logreg_proba = clf.predict_proba(X_residual_test)[:,1]

plot_roc(y_residual_test,y_pred_logreg_proba)

plt.figure()
sns.distplot(y_pred_logreg_proba[y_residual_test==1],norm_hist=True,kde=False,color='red')
sns.distplot(y_pred_logreg_proba[y_residual_test==0],norm_hist=True,kde=False,color='blue')
plt.legend(['failure residuals','non failure residuals'])
plt.title('distribution of logistic regression probability')

In [ ]:
y_pred_threshold_logreg = np.where(y_pred_logreg_proba > .25, 1, 0)

print('accuracy = {:.3f}'.format(accuracy_score(y_residual_test,y_pred_threshold_logreg)))
print('precision = {:.3f}'.format(precision_score(y_residual_test,y_pred_threshold_logreg)))
print('recall = {:.3f}'.format(recall_score(y_residual_test,y_pred_threshold_logreg)))
print('f1 = {:.3f}\n\n'.format(f1_score(y_residual_test,y_pred_threshold_logreg)))
print(confusion_matrix(y_residual_test,y_pred_threshold_logreg))

# Try decision tree on reconstruction error

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_dt = DecisionTreeClassifier()
clf_dt.fit(X_residual_train,y_residual_train)
y_pred_dt = clf_dt.predict(X_residual_test)

print('accuracy = {:.3f}'.format(accuracy_score(y_residual_test,y_pred_dt)))
print('precision = {:.3f}'.format(precision_score(y_residual_test,y_pred_dt)))
print('recall = {:.3f}'.format(recall_score(y_residual_test,y_pred_dt)))
print('f1 = {:.3f}\n\n'.format(f1_score(y_residual_test,y_pred_dt)))
print(confusion_matrix(y_residual_test,y_pred_dt))

# Try SVC on reconstruction error

In [ ]:
from sklearn.svm import SVC

clf_svc = SVC(C=5)
clf_svc.fit(X_residual_train,y_residual_train)
y_pred_svc = clf_svc.predict(X_residual_test)

print('accuracy = {:.3f}'.format(accuracy_score(y_residual_test,y_pred_svc)))
print('precision = {:.3f}'.format(precision_score(y_residual_test,y_pred_svc)))
print('recall = {:.3f}'.format(recall_score(y_residual_test,y_pred_svc)))
print('f1 = {:.3f}\n\n'.format(f1_score(y_residual_test,y_pred_svc)))
print(confusion_matrix(y_residual_test,y_pred_svc))

## T-sne on reconstruction errors

In [ ]:
perplexity_list = [5,20,45]

for p in perplexity_list:
    tsne = TSNE(n_components=2,perplexity=p)
    tsne_output = tsne.fit_transform(residuals_test)
    tsne_output1 = pd.Series(tsne_output[:,0],index=residuals_test.index)
    tsne_output2 = pd.Series(tsne_output[:,1],index=residuals_test.index)

    plt.figure(figsize=(8,8))
    plt.scatter(tsne_output1[nominal_idx],tsne_output2[nominal_idx],color='blue')
    plt.scatter(tsne_output1[early1_idx],tsne_output2[early1_idx],color='red')
    plt.legend(['nonfailure','early1'])
    plt.title('tsne plot 1 vs 2, perplexity = {}'.format(p))

### Try K means clustering on reconstruction error 